# Optimization

**Table of contents**<a id='toc0_'></a>    
- 1. [Setup](#toc1_)    
- 2. [Golden Section Search](#toc2_)    
- 3. [Newton-Raphson](#toc3_)    
- 4. [QuantEcon: Nelder-Mead](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

This notebooks shows how to use the **numerical optimizers** in **consav**.

## 1. <a id='toc1_'></a>[Setup](#toc0_)

In [1]:
%load_ext autoreload
%autoreload 2

import time
from numba import njit
import numpy as np

## 2. <a id='toc2_'></a>[Golden Section Search](#toc0_)

The **golden_section_search** module provides a **Numba JIT** compilled one-dimensional **optimizer function** for a user-defined Numba JIT compilled function with abirtrary number of fixed inputs.

In [2]:
from consav import golden_section_search

# a. target function
@njit
def f(x,alpha,beta):
    return (x-alpha)*x*(x+beta)**2

# b. call optimizer
low = 0.0
high = 4.0
tol = 1e-5
alpha = 2.0
beta = 1.2
result = golden_section_search.optimizer(f,low,high,args=(alpha,beta),tol=tol)

print(f'result = {result:.8f}  with f = {f(result,alpha,beta):.8}')

result = 1.34582467  with f = -5.7061033


## 3. <a id='toc3_'></a>[Newton-Raphson](#toc0_)

The **newton_raphson** module provides a **Numba JIT** compilled multi-dimensional **optimizer function** for a user-defined Numba JIT compilled function with abirtrary number of fixed inputs.

In [3]:
from consav import newton_raphson

# a. target function
@njit
def f(x,low,high,alpha,beta):        

    penalty = 0.0
    if x[0] < low:
        penalty += 1000*(low-x[0])
    elif x[0] > high:
        penalty += 1000*(x[0]-high)
        
    return (x[0]-alpha)*x[0]*(x[0]+beta)**2 + penalty

# c. call optimizer
alpha = 2.0
beta = 1.2
low = 0.0
high = 4.0
guess = np.array([1.3])
result = newton_raphson.optimizer(f,guess,args=(low,high,alpha,beta))

print(f'result = {result[0]:.8f} with f = {f(result,low,high,alpha,beta):.8}')

c:\users\gmf123.unicph\documents\repositories\consumptionsaving\consav\newton_raphson.py:50: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  num_grad(obj,x_min,grad_step,f_min,grad,x_grad,*args)
c:\users\gmf123.unicph\documents\repositories\consumptionsaving\consav\newton_raphson.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  num_hess(obj,x_min,grad_step,grad,hess,x_grad,x_hess,grad_hess,*args)
c:\users\gmf123.unicph\documents\repositories\consumptionsaving\consav\newton_raphson.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  num_hess(obj,x_min,grad_step,grad,hess,x_grad,x_hess,grad_hess,*args)
C:\Users\gmf123.UNICPH\AppData\Local\anaconda3\Lib\site-packages\numba\core\utils.py:554: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",


result = 1.34581691 with f = -5.7061033


**Note:** It is unresolved why the matrix multiplication raises an performance warning in some versions of numba.

## 4. <a id='toc4_'></a>[QuantEcon: Nelder-Mead](#toc0_)

In [4]:
import quantecon as qe

In [5]:
@njit
def g(x,low,high,alpha,beta):
    return -f(x,low,high,alpha,beta) # to maximize

bounds = np.zeros((1,2))
bounds[0,0] = low
bounds[0,1] = high
results = qe.optimize.nelder_mead(g,guess,bounds=bounds,args=(low,high,alpha,beta))
print(results)

results(x=array([1.34582214]), fun=5.706103284678152, success=True, nit=14, final_simplex=array([[1.34582214],
       [1.34582611]]))
